## Сбор данных

Данные – информация о популярных каналах стримингового серсива Twitch.tv
Для сбора данных необходимо два источника информации:
1. [Socialblade](https://socialblade.com/twitch/top/500/followers)– сайт, предоставляющий списки наиболее популярных стримеров (по количеству подписчиков и по количеству просмотров канала) 
2. [Twitch](https://www.twitch.tv) (данные собираются посредством [API](https://dev.twitch.tv))

In [ ]:
# загрузка библиотек
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd

Для начала, сбор информации с Socialblade (названия каналов):

In [1]:
# сначала со вкладки, на которой представлены каналы с наибольшим числом подписчиков:
sbl = requests.get("https://socialblade.com/twitch/top/500/followers")
soup = bs(sbl.text, 'html.parser')

num = np.arange(2, 2500, 5)
d = []

for i in range(500):
    d.append({'name': soup.find_all(class_='table-cell')[num[0][i]].get_text()})
    df1 = pd.DataFrame(d)


# то же самое для каналов, отсортированных по количеству просмотров:
sbl2 = requests.get("https://socialblade.com/twitch/top/500/channelviews")
soup2 = bs(sbl2.text, 'html.parser')

d2 = []

for i in range(500):
    d2.append({'name': soup2.find_all(class_='table-cell')[num[0][i]].get_text()})
    df2 = pd.DataFrame(d2)
    
# Лично мне удобнее полученные списки привести в табличный формат (pd) для дальнейших объединения и фильтрации 
frames = [df1, df2]
df = pd.concat(frames)

df=df.reset_index()
df.drop(['index'], axis = 1, inplace = True)
df=df.drop_duplicates() # один канал тможет находиться в обоих списках
df=df.reset_index()
df.drop(['index'], axis = 1, inplace = True)

Теперь можно перейти ко сбору данных с самого Twitch.tv

In [8]:
twitch='https://api.twitch.tv/kraken/channels/'
headers = {"Client-ID": "fq4s3rmzc9erhbws2n3z9u20kqf1rh"}
urls=[]
urls.append(twitch + df.name)

twt = requests.get(urls[0][0],headers=headers)
twt.json()["_links"]

{'self': 'https://api.twitch.tv/kraken/channels/ninja',
 'follows': 'https://api.twitch.tv/kraken/channels/ninja/follows',
 'commercial': 'https://api.twitch.tv/kraken/channels/ninja/commercial',
 'stream_key': 'https://api.twitch.tv/kraken/channels/ninja/stream_key',
 'chat': 'https://api.twitch.tv/kraken/chat/ninja',
 'features': 'https://api.twitch.tv/kraken/channels/ninja/features',
 'subscriptions': 'https://api.twitch.tv/kraken/channels/ninja/subscriptions',
 'editors': 'https://api.twitch.tv/kraken/channels/ninja/editors',
 'teams': 'https://api.twitch.tv/kraken/channels/ninja/teams',
 'videos': 'https://api.twitch.tv/kraken/channels/ninja/videos'}

Как видно выше, API позволяет собрать данные о канале по многим разделам (информация о чате/видео/командах и тд), однако, не все ссылки являются действующими до сих пор.
Тем не менее, наиболее важные ссылки всё ещё позволяют найти необходимые данные:

In [9]:
s=[]
for k in range(734): # После удаления дублирующихся данных осталось 734 канала 
    v = str('https://api.twitch.tv/kraken/channels/' + df.name[k] + '/videos?broadcasts=true') # прошлые прямые трансляции
    twt1 = requests.get(v, headers=headers)
    n = str('https://api.twitch.tv/kraken/channels/' + df.name[k]) # общие сведения о канале
    twt2 = requests.get(n,headers=headers)
    t = str('https://api.twitch.tv/kraken/channels/' + df.name[k] + '/teams') # сообщества или т.н. "команды"
    twt3 = requests.get(t,headers=headers)
    h = str('https://api.twitch.tv/kraken/channels/' + df.name[k] + '/videos') # "яркие" моменты канала
    twt4 = requests.get(h,headers=headers)
    if '_total' in twt1.json().keys() and 'broadcaster_language' in twt2.json():
            s.append({'name': df.name[k], 'views': twt2.json()['views'], 'followers': twt2.json()['followers'], 'broadcasts_num': twt1.json()['_total'], 'highlights_num': twt4.json()['_total'], 'language': twt2.json()['broadcaster_language'],
                  'in_partnership': twt2.json()['partner'], 'teams': len(twt3.json()['teams'])})
    else:
        pass


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [17]:
# Создание датафрейма
df=pd.DataFrame(s)
df=df[df.broadcasts_num != 0] # Socialblade предоставляет данные о каналах так, что один и тот же может появиться несколько раз, 
# если ранее он был переименован. В переименованных версиях, как правило, отсутствуют данные о предыдущих трансляциях
df=df.reset_index()
df.drop(['index'], axis = 1, inplace = True)

Одна из идей исследования заключалась в важности популярной игры канала. Поэтому, эти данные также были собраны:

In [23]:
v=[]
for n in range(546): # Часть отсеялась на предыдущем этапе
    for i in np.arange(0,500,100): # максимальное число записей, которые мы готовы принять – последние 500 трансляций
        link = str('https://api.twitch.tv/kraken/channels/' + df.name[n] + '/videos?broadcasts=true&limit=100&login=ninja&offset='+ str(i))
        req=requests.get(link,headers=headers)
        for k in np.arange(0,100,1):
            try:
                v.append({'name': df.name[n],'preval_game': req.json()['videos'][k]['game']})
            except Exception:
                pass

In [24]:
# Подсчёт и отбор наиболее популярной игры для каждого канала

d = pd.DataFrame(v)
d['count'] = d.groupby(['name','preval_game'])['preval_game'].transform('count')
d=d.drop_duplicates()
d=d[d['count'] == d.groupby(['name'])['count'].transform(max)]
d=d.reset_index()
d.drop(['index','count'], axis = 1, inplace = True)

,name,preval_game
0,ninja,Fortnite
1,shroud,PLAYERUNKNOWN'S BATTLEGROUNDS
2,tsm_myth,Fortnite
3,summit1g,Destiny 2
4,tfue,Fortnite
5,riotgames,League of Legends
6,timthetatman,IRL
7,drdisrespectlive,Call of Duty: Black Ops 4
8,Dakotaz,Fortnite
9,tsm_daequan,Fortnite


In [25]:
# Объединение датасетов

result = pd.merge(df, d, how='left', on=['name'])
result

,broadcasts_num,followers,highlights_num,in_partnership,language,name,teams,views,preval_game
0,455,11927087,247,True,en,ninja,1,356456708,Fortnite
1,870,4670969,35,True,en,shroud,0,225916798,PLAYERUNKNOWN'S BATTLEGROUNDS
2,60,4439305,106,True,en,tsm_myth,2,62075985,Fortnite
3,66,3320786,268,True,en,summit1g,2,245212181,Destiny 2
4,100,3265483,152,True,en,tfue,1,48289755,Fortnite
5,1505,3230263,332,True,en,riotgames,2,1063926023,League of Legends
6,84,2933071,5,True,en,timthetatman,1,106450301,IRL
7,47,2859376,41,True,en,drdisrespectlive,2,99107457,Call of Duty: Black Ops 4
8,57,2813406,1,True,en,Dakotaz,1,65129357,Fortnite
9,55,2757590,0,True,en,tsm_daequan,1,32257129,Fortnite


In [26]:
df.to_csv('twitchdata.csv')